## Inspecting the memory layout of a Python list

based on https://jakevdp.github.io/blog/2014/05/09/why-python-is-slow/

In [ ]:
import sys
import ctypes
import numpy

In [ ]:
x=[1,[2,3]]
print(type(x), type(x[0]))
x=1.2
sys.getsizeof(x)

In [ ]:
# what is the data type for integers?
N=10000
print(N, "has type", type(N))

In [ ]:
# Integers are dynamically sized! Check its size in bytes:
a=46560000000000000000000000000000000000000

print("a has type", type(a))

print("bytes used to store 1:", sys.getsizeof(1))
print("bytes used to store a:", sys.getsizeof(a))

In [ ]:
class IntStruct(ctypes.Structure): # PyLongObject
    _fields_ = [("ob_refcnt", ctypes.c_long),
                ("ob_type", ctypes.c_void_p),
                ("ob_size", ctypes.c_ulong),
                ("ob_digit", ctypes.c_long)]

This is how an int is represented in memory:

In [ ]:
x = 1000000 # change to a really large number
y = x

print("ids:", id(x), id(y), id(100), id(1234567))

info=IntStruct.from_address(id(x))
print("refcount =", info.ob_refcnt, 
      "type =", info.ob_type,
      "size =", info.ob_size, 
      "digits =", info.ob_digit)

How about a List?

In [ ]:
class ListStruct(ctypes.Structure): # struct PyListObject
    _fields_ = [("ob_refcnt", ctypes.c_long),
                ("ob_type", ctypes.c_void_p),
                ("ob_size", ctypes.c_ulong),
                ("ob_item", ctypes.c_long),  # PyObject** pointer cast to long
                ("allocated", ctypes.c_ulong)]

x = [1,[2,3]]

info = ListStruct.from_address(id(x))
print("refcount =", info.ob_refcnt, 
      "type =", info.ob_type,
      "size =", info.ob_size, 
      "item =", info.ob_item,
      "allocated = ", info.allocated)


What happens if we append to the list x?

In [ ]:
y = x # this increases reference count
x.append(4) # this will resize the array
info = ListStruct.from_address(id(x))
print("refcount =", info.ob_refcnt, 
      "type =", info.ob_type,
      "size =", info.ob_size, 
      "item =", info.ob_item,
      "allocated = ", info.allocated)
